# Example: Solving Ordinary Differential Equations

In this notebook we will use Python to solve differential equations numerically.

In [ ]:
// Import the required modules
#r "nuget: Plotly.NET.Interactive, 3.0.2"

#I "./bin/Release/net6.0"
#r "AltaxoCore.Redist.dll"
#r "FSOde.dll"

// ... and open the required modules
open FsODE
open Plotly.NET

Installed Packages Plotly.NET.Interactive, 3.0.2

Loading extensions from `Plotly.NET.Interactive.dll`

### First-order equations
Let's try a first-order ordinary differential equation (ODE), say:
$$\quad 
\frac{dy}{dx} + y = x, \quad \quad y(0) = 1.
$$
This has a closed-form solution
$$\quad
y = x - 1 + 2e^{-x}
$$
(Exercise: Show this, by first finding the integrating factor.)

We are going to solve this numerically.

In [ ]:
// First set the model context that remembers the solver method and it´s optiony 
let modelContext = //OdeContext()
    OdeSolverMethod.RK546M //RK547M()
    |> OdeContext

// Define a function which calculates the derivative
// Simple version (no vector)
let dy_dx : SimpleModel =     
    fun y x ->            
        x - y

// Initial condition
let x0 = 0.0  
let y0 = 1.0  

In [ ]:
// Simulate the model function
let sim_dy_dx = 
    modelContext.OdeInt(x0,y0,dy_dx)
    |> SolPoints.take 10 
    |> SolPoints.memorize

// Plot the numerical solution
sim_dy_dx
|> SolPoints.toPoints 1
|> Chart.Point

<!-- Plotly chart will be drawn inside this DIV -->

Compare the numerical solution with the analytical solution by showing both on the same plot

In [ ]:
let y_exact =
    fun x -> 
        x - 1. + 2. * exp(-x)

[
    sim_dy_dx
    |> SolPoints.toPoints 1
    |> Chart.Spline 
    |> Chart.withTraceInfo("simulation");
    
    sim_dy_dx
    |> SolPoints.map (fun p -> (p.x, y_exact p.x))
    |> Chart.Point
    |> Chart.withTraceInfo("exact");

]
|> Chart.combine

<!-- Plotly chart will be drawn inside this DIV -->

Now take a look at the difference between the two series:

In [ ]:
let errors = 
    sim_dy_dx
    |> SolPoints.map (fun p -> p.Y[0] - (y_exact p.x))
    
Chart.Spline ([1..10], errors)
|> Chart.withXAxisStyle("x")
|> Chart.withYAxisStyle("error")
|> Chart.withTitle("Error in numerical integration")

<!-- Plotly chart will be drawn inside this DIV -->

Exercise: Experiment with the options of "odeint" to improve the accuracy of the integration.

## Second-order ordinary differential equations
Suppose we have a second-order ODE such as a damped simple harmonic motion equation,
$$
\quad y'' + 2 y' + 2 y = \cos(2x), \quad \quad y(0) = 0, \; y'(0) = 0
$$
We can turn this into two first-order equations by defining a new depedent variable. For example,
$$
\quad z \equiv y' \quad \Rightarrow \quad z' + 2 z + 2y = \cos(2x), \quad z(0)=y(0) = 0.
$$
We can solve this system of ODEs using "odeint" with lists, as follows:

In [ ]:
// Define a function which calculates the derivative
let dU_dx : Model = 
    // Here U is a vector such that y=U[0] and z=U[1]. This function should return [y', z']
    fun U x ->
        let y = U[0]
        let z = U[1]

        [| z; -2.*z - 2.*y + cos(2.*x)|]

// Initilal condition
let u0 = [| 0.; 0. |]


modelContext.OdeInt(1.,u0,dU_dx)
|> SolPoints.take 30
|> SolPoints.toPoints 1
|> Chart.Spline
|> Chart.withXAxisStyle("x")
|> Chart.withYAxisStyle("y")
|> Chart.withTitle("Damped harmonic oscillator")


<!-- Plotly chart will be drawn inside this DIV -->

## Predator-Prey Equations
Also known as <a href="http://en.wikipedia.org/wiki/Lotka-Volterra_equation">Lotka-Volterra equations</a>, the predator-prey equations are a pair of first-order non-linear ordinary differential equations. They represent a simplified model of the change in populations of two species which interact via predation. For example, foxes (predators) and rabbits (prey). Let $x$ and $y$ represent rabbit and fox populations, respectively. Then
\begin{align}
 \frac{dx}{dt} &= x (a - b y) \\
 \frac{dy}{dt} &= -y (c - d x) 
\end{align}
Here $a$, $b$, $c$ and $d$ are parameters, which are assumed to be positive. 

In [ ]:
let a,b,c,d = 1.,1.,1.,1.

// Define a function which calculates the derivative
let dP_dt : Model = 
    fun P t ->
        let x = P[0] // predators
        let y = P[1] // prey

        [| x*(a - b*y); -y*(c - d*x) |]

let P0 = [|1.5; 1.0|]

let Ps = 
    modelContext.OdeInt(0.,P0,dP_dt)
    |> SolPoints.take 30
    |> SolPoints.memorize 
    
let predators = Ps |> SolPoints.toPoints 1
let prey = Ps |> SolPoints.toPoints 2

In [ ]:
[
    predators
    |> Chart.Spline 
    |> Chart.withTraceInfo("Foxes");
    
    prey
    |> Chart.Spline
    |> Chart.withTraceInfo("Rabbits");

]
|> Chart.combine
|> Chart.withXAxisStyle("time")
|> Chart.withYAxisStyle("population")

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let predatorsY = Ps |> SolPoints.toY 0
let preyY = Ps |> SolPoints.toY 1

Chart.Point(preyY,predatorsY)
|> Chart.withXAxisStyle("Rabbits")
|> Chart.withYAxisStyle("Foxes")
|> Chart.withTitle("Rabbits vs Foxes")

<!-- Plotly chart will be drawn inside this DIV -->

The plot above illustrates that the system is periodic. Let's plot a few more curves in the phase space.

In [ ]:
[
    for r in [1.0 ..0.2.. 3.0] do
        let P0 = [|r; 1.|]
        let Ps = 
            modelContext.OdeInt(0.,P0,dP_dt)
            |> SolPoints.take 20
            |> SolPoints.memorize

        let predatorsY = Ps |> SolPoints.toY 0
        let preyY = Ps |> SolPoints.toY 1

        Chart.Spline(preyY,predatorsY)
    
]
|> Chart.combine        
|> Chart.withXAxisStyle("Rabbits")
|> Chart.withYAxisStyle("Foxes")
|> Chart.withTitle("Rabbits vs Foxes")


<!-- Plotly chart will be drawn inside this DIV -->